In [1]:
!pip install transformers -q
!pip install wandb -q
!pip install SentencePiece -q

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import TFMT5Model, T5Tokenizer , MT5ForConditionalGeneration
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from transformers import AutoTokenizer, MT5Model
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [5]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        labels = y[:, 1:].clone().detach()
        labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=labels)
        loss = outputs[0]
        
        if _%10 == 0:
            wandb.log({"Training Loss": loss.item()})

        if _%10==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [6]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("/kaggle/input/hehehe/test.csv" ,encoding="utf8")

In [10]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [15]:
df.columns = ['ctext']
df['ctext'] = "summarize: "+df['ctext']

In [16]:
df

,ctext
0,summarize: ‘वारिस पंजाब दे’ संगठन के चीफ अमृतप...
1,summarize: कांग्रेस नेता राहुल गांधी जल्द ही स...
2,summarize: संसद परिसर में मंगलवार को आम आदमी प...
3,summarize: राष्ट्रीय लोकतांत्रिक पार्टी (RLP) ...
4,summarize: मैं कश्मीर से हूं। बचपन में मुझे बत...
...,...
2995,summarize: कर्नाटक में एक महिला के खेत से 2.5 ...
2996,summarize: सुप्रीम कोर्ट ने मंगलवार को भोपाल ग...
2997,summarize: नीतू सिंह के पुराने इंटरव्यू की न्य...
2998,summarize: बीआर चोपड़ा की महाभारत में युधिष्ठि...


In [18]:
import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, ' ', data)

def preprocess_tokenize(text):
      # for removing punctuation from sentencesc
    text = str(text)
    text = re.sub(r'(\d+)', r'', text)
    
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    text = text.replace('\t', ' ')
    text = text.replace('\u200d', '')
    text=re.sub("(__+)", ' ', str(text)).lower()   #remove _ if it occors more than one time consecutively
    text=re.sub("(--+)", ' ', str(text)).lower()   #remove - if it occors more than one time consecutively
    text=re.sub("(~~+)", ' ', str(text)).lower()   #remove ~ if it occors more than one time consecutively
    text=re.sub("(\+\++)", ' ', str(text)).lower()   #remove + if it occors more than one time consecutively
    text=re.sub("(\.\.+)", ' ', str(text)).lower()   #remove . if it occors more than one time consecutively
    text=re.sub(r"[<>()|&©@#ø\[\]\'\",;:?.~*!]", ' ', str(text)).lower() #remove <>()|&©ø"',;?~*!
    text = re.sub(r"[‘’।:]", " ", str(text)) #removing other special characters
    text = re.sub("([a-zA-Z])",' ',str(text)).lower()
    text = re.sub("(\s+)",' ',str(text)).lower()
    text = remove_emojis(text)
    return text

In [19]:
df['ctext']= [preprocess_tokenize(i) for i in df['ctext']]


In [21]:
df['ctext']="summarize: "+df['ctext']
df

,ctext
0,summarize: वारिस पंजाब दे संगठन के चीफ अमृतपा...
1,summarize: कांग्रेस नेता राहुल गांधी जल्द ही ...
2,summarize: संसद परिसर में मंगलवार को आम आदमी ...
3,summarize: राष्ट्रीय लोकतांत्रिक पार्टी के प्...
4,summarize: मैं कश्मीर से हूं बचपन में मुझे बत...
...,...
2995,summarize: कर्नाटक में एक महिला के खेत से लाख...
2996,summarize: सुप्रीम कोर्ट ने मंगलवार को भोपाल ...
2997,summarize: नीतू सिंह के पुराने इंटरव्यू की न्...
2998,summarize: बीआर चोपड़ा की महाभारत में युधिष्ठ...


In [20]:
from huggingface_hub import notebook_login

notebook_login()

In [22]:
df['text']='x'
df

,ctext,text
0,summarize: वारिस पंजाब दे संगठन के चीफ अमृतपा...,x
1,summarize: कांग्रेस नेता राहुल गांधी जल्द ही ...,x
2,summarize: संसद परिसर में मंगलवार को आम आदमी ...,x
3,summarize: राष्ट्रीय लोकतांत्रिक पार्टी के प्...,x
4,summarize: मैं कश्मीर से हूं बचपन में मुझे बत...,x
...,...,...
2995,summarize: कर्नाटक में एक महिला के खेत से लाख...,x
2996,summarize: सुप्रीम कोर्ट ने मंगलवार को भोपाल ...,x
2997,summarize: नीतू सिंह के पुराने इंटरव्यू की न्...,x
2998,summarize: बीआर चोपड़ा की महाभारत में युधिष्ठ...,x


In [23]:
wandb.init(project="MT5 Hindi Predicitons Final Try")

config = wandb.config         
config.TRAIN_BATCH_SIZE = 2  
config.VALID_BATCH_SIZE = 2    
config.TRAIN_EPOCHS = 8        
config.VAL_EPOCHS = 1 
config.LEARNING_RATE = 5e-4    
config.SEED = 42               
config.MAX_LEN = 512
config.SUMMARY_LEN = 75


torch.manual_seed(config.SEED) 
np.random.seed(config.SEED)
torch.backends.cudnn.deterministic = True

val_dataset = df

wandb: Currently logged in as: saliq7 (nitk-ai). Use `wandb login --relogin` to force relogin


In [24]:
val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

# parameters fo dataloaders
train_params = {
    'batch_size': config.TRAIN_BATCH_SIZE,
    'shuffle': True,
    'num_workers': 0
    }

val_params = {
    'batch_size': config.VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
    }

val_loader = DataLoader(val_set, **val_params)

In [25]:
model = MT5ForConditionalGeneration.from_pretrained("saliq7/hindi_mt5_finaltry")
model = model.to(device)

In [34]:
for epoch  in range(config.VAL_EPOCHS):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'ans':predictions,'ref':actuals})
    final_df.to_csv('predictions_final_test2.csv')
    print('Output Files generated for review')

Completed 0
Completed 100
Completed 200
Completed 300
Completed 400
Completed 500
Completed 600
Completed 700
Completed 800
Completed 900
Completed 1000
Completed 1100
Completed 1200
Completed 1300
Completed 1400
Output Files generated for review


In [35]:
df1 = pd.read_csv('/kaggle/working/predictions_final_test2.csv')

In [36]:
df1

,Unnamed: 0,ans,ref
0,0,और,x
1,1,और,x
2,2,और,x
3,3,NaN,x
4,4,और,x
...,...,...,...
2995,2995,NaN,x
2996,2996,और,x
2997,2997,रहा,x
2998,2998,और,x


In [37]:
df['ctext'][0]

'summarize:  वारिस पंजाब दे संगठन के चीफ अमृतपाल सिंह को पंजाब पुलिस ने रविवार सुबह अरेस्ट कर लिया मार्च को पुलिस ने अमृतपाल की गिरफ्तारी के लिए उसके गांव जल्लूपुर खेड़ा में दबिश दी थी पर वह भाग निकला था दिन बाद उसे मोगा के रोडे गांव से गिरफ्तार किया गया है सिख कट्टरपंथी जरनैल सिंह भिंडरांवाले इसी गांव का रहने वाला था सितंबर को यहीं अमृतपाल को वारिस पंजाब दे का प्रमुख घोषित किया गया था अमृतपाल पर अजलाना थाने पर हमला कर साथी को छुड़ाने का आरोप है जल्लूपुर खेड़ा गांव में किलेनुमा घर में रहने वाला अमृतपाल नशा मुक्ति केंद्र चला रहा था जहां मरीजों को फ्री में दवा और खाना मिलता है अमृृतपाल खुद को गर्व से खालिस्तान समर्थक बताता है खालिस्तान यानी खालसा या सिखों का अलग देश ये दावा भी करता है कि लोग खालिस्तान के समर्थन में हैं फरवरी को अजनाला थाने पर हमले के बाद भास्कर ने अमृतपाल के गांव से रिपोर्ट की थी उसका इंटरव्यू भी लिया ये इंटरव्यू मार्च को पब्लिश हुआ था आज उसे दोबारा पब्लिश कर रहे हैं… सवाल जिस रास्ते पर आप चल रहे हैं उससे क्या हासिल करना चाहते हैं जवाब मैं ड्रग्स और पंजाब के मुद्दा उठाता ह

In [38]:
string = input("Enter text")


Enter text वारिस पंजाब दे संगठन के चीफ अमृतपाल सिंह को पंजाब पुलिस ने रविवार सुबह अरेस्ट कर लिया मार्च को पुलिस ने अमृतपाल की गिरफ्तारी के लिए उसके गांव जल्लूपुर खेड़ा में दबिश दी थी पर वह भाग निकला था दिन बाद उसे मोगा के रोडे गांव से गिरफ्तार किया गया है सिख कट्टरपंथी जरनैल सिंह भिंडरांवाले इसी गांव का रहने वाला था सितंबर को यहीं अमृतपाल को वारिस पंजाब दे का प्रमुख घोषित किया गया था अमृतपाल पर अजलाना थाने पर हमला कर साथी को छुड़ाने का आरोप है जल्लूपुर खेड़ा गांव में किलेनुमा घर में रहने वाला अमृतपाल नशा मुक्ति केंद्र चला रहा था जहां मरीजों को फ्री में दवा और खाना मिलता है अमृृतपाल खुद को गर्व से खालिस्तान समर्थक बताता है खालिस्तान यानी खालसा या सिखों का अलग देश ये दावा भी करता है कि लोग खालिस्तान के समर्थन में हैं फरवरी को अजनाला थाने पर हमले के बाद भास्कर ने अमृतपाल के गांव से रिपोर्ट की थी उसका इंटरव्यू भी लिया ये इंटरव्यू मार्च को पब्लिश हुआ था आज उसे दोबारा पब्लिश कर रहे हैं… सवाल जिस रास्ते पर आप चल रहे हैं उससे क्या हासिल करना चाहते हैं जवाब मैं ड्रग्स और पंजाब के मुद्दा उठाता हूं

In [41]:
data = [string] 
  
# Create the pandas DataFrame with column name is provided explicitly 
df_temp = pd.DataFrame(data, columns=['ctext'])

In [43]:
df_temp['ctext']= "summaize: "+df_temp['ctext']

In [45]:
df_temp['text']=""
df_temp

,ctext,text
0,summaize: वारिस पंजाब दे संगठन के चीफ अमृतपाल ...,


In [51]:
Test = CustomDataset(df_temp, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)


Test_loader = DataLoader(Test, {
    'batch_size': 2000,
    'shuffle': False,
    'num_workers': 0
    } )

predictions, actuals = validate(epoch, tokenizer, model, device, Test_loader)
print('Generated Text' ,predictions)
print('Actual Text',actuals)

ValueError: batch_size should be a positive integer value, but got batch_size={'batch_size': 2000, 'shuffle': False, 'num_workers': 0}